# Argentina Programa 4.0 / UNSAM - Módulo 3 - Aprendizaje Automático

## Examen parcial

Para empezar, te pedimos que corras la siguiente celda preparatoria.

Es importante que uses Colab o que tengas instaladas las mismas versiones de los paquetes `numpy`, `scikit-learn` y `xgboost`. El código de esta celda verifica esto.

In [3]:
import importlib

def check_version(module, targetversion):
    try:
        mod = importlib.import_module(module)
        assert mod.__version__ == targetversion, "La versión de {} debe ser {}; vos tenés la {}.".format(module, targetversion, mod.__version__)
    except ModuleNotFoundError:
        raise ModuleNotFoundError("Falta instalar ", module)

def check_dni(dni1, dni2):
    if dni1 != dni2:
        raise ValueError(f"Los números ingresados ({dni1} y {dni2}) no coinciden. Fijate mejor.")
    else:
        dni = dni1
    return dni

# Check versions de los módulos clave
check_version('numpy', '1.23.5')
check_version('sklearn', '1.2.2')
check_version('xgboost', '1.7.6')

Las celdas a continuación se encargan de generar un **dataset** y **consigna** únicos para este examen.

Para eso, hace falta que completes tu DNI o pasaporte en la celda de abajo (sin puntos, solo números) y que corras las celdas que siguen.

>**Es importante que uses el número con que te inscribiste a Argentina Programa, porque vamos a usar eso para realizar la corrección. No pongas un número cualquiera.**

In [4]:
dni1 = ...

Para verificar, volvé a anotar el número de tu documento. La función de abajo verifica que esté bien ingresado. Si no hay error, podés seguir adelante

In [7]:
dni2 = ...

dni = check_dni(dni1, dni2)

# ...MAGIA...

In [62]:
from sklearn.datasets import make_moons

X, t = make_moons(n_samples=200, random_state=dni, noise=0.5)

## Ejercicio 1

Usar el dataset generado arriba para entrenar un SVC con kernel polinómico (`kernel='poly'`).

Optimizar los hiperparámetros usando `GridSearchCV`:

grado del polinomio (`degree`), probando los valores [2, 5, 7].

constante de regularización (`C`), probando los valores [1, 30.0, 180.0].

Para el resto de los parámetros, usar los valores por defecto.

Como función de scoring usar `roc_auc`.

Usar cinco (5) cortes de validación cruzada.

¡Importante! En todos los casos fijar `random_state=42`.

Reportar:

1. Valor optimizado de grado del polinomio (`degree`)

2. Valor optimizado de constante de regularización (`C`)

3. Valor óptimo de `roc_auc` obtenido con `GridSearchCV`


In [11]:
from sklearn.svm import SVC

misvc = SVC(kernel='poly', random_state=42)

In [13]:
from sklearn.model_selection import GridSearchCV

migscv = GridSearchCV(estimator = misvc,
                      param_grid = {'degree': [2, 5, 7],
                                    'C': [1, 30.0, 180.0]},
                    cv = 5,
                    scoring = 'roc_auc',
                    n_jobs=-1)

In [14]:
migscv.fit(X, t)

GridSearchCV(cv=5, estimator=SVC(kernel='poly', random_state=42), n_jobs=-1,
             param_grid={'C': [1, 30.0, 180.0], 'degree': [2, 5, 7]},
             scoring='roc_auc')

In [17]:
migscv.best_params_

{'C': 180.0, 'degree': 5}

In [20]:
round(migscv.best_score_, 2)

0.96

## Ejercicio 2

Usar el dataset generado arriba para entrenar un XGBoostClassifier.

Optimizar los hiperparámetros usando `GridSearchCV`:

número de estimadores (`n_estimators`), probando los valores [80, 110, 200].

'profundidad máxima (`max_depth`), probando los valores [3, 7, 10].

Para el resto de los parámetros, usar los valores por defecto.

Como función de scoring usar `accuracy`.

Usar cinco cortes de validación cruzada.

¡Importante! En todos los casos fijar `random_state=42`.

Reportar:

1. Valor optimizado de número de estimadores (`n_estimators`)

2. Valor optimizado de profundidad máxima (`max_depth`)

3. Valor óptimo de `accuracy` obtenido con `GridSearchCV`

In [63]:
from xgboost import XGBClassifier

mixgbc = XGBClassifier(random_state=42)

In [69]:
from sklearn.model_selection import GridSearchCV

migscv = GridSearchCV(estimator=mixgbc,
             param_grid = {'n_estimators': [20, 30, 35, 40, 50, 100],
                           'max_depth': [1, 2, 3, 7]},
             cv = 5,
             scoring = 'roc_auc',
             n_jobs=-1)

In [70]:
migscv.fit(X, t)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 7],
                         'n_estimators': [20, 30, 35, 40, 50, 100]},
             scoring='roc_auc')

In [71]:
migscv.best_params_

{'max_depth': 1, 'n_estimators': 50}

In [72]:
round(migscv.best_score_, 2)

0.88

-----

## Generemos un conjunto de testeo (no va para el examen)

In [80]:
X_test, t_test = make_moons(n_samples=500, random_state=43243256, noise=0.5)

In [81]:
best = migscv.best_estimator_

In [82]:
y_test=best.predict(X_test)


In [83]:
from sklearn.metrics import accuracy_score, roc_auc_score

print(accuracy_score(t_test, y_test))

0.804


In [84]:
print(roc_auc_score(t_test, y_test))

0.8040000000000002
